In [1]:
%matplotlib inline
!pip install dask_ml

You are using pip version 9.0.3, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from dask.distributed import Client
client = Client()  # start a local Dask client

import dask_ml.joblib
from sklearn.externals import joblib

with joblib.parallel_backend('dask'):
    #Load and split the data
    training_data = pd.read_csv("train.csv")
    testing_data = pd.read_csv("test.csv")
    costa_rica_data = training_data.drop(['Target'], axis=1)
    costa_rica_target = training_data['Target']

    #Clean the data to either replace or remove string columns
    costa_rica_data.select_dtypes(exclude=[np.number]).head()
    costa_rica_data = costa_rica_data.select_dtypes(include=[np.number], exclude=[np.object]).fillna(0)
    
    X_train, X_test, y_train, y_test = train_test_split(costa_rica_data, costa_rica_target, test_size= 0.2, random_state=42)
    clf = DecisionTreeClassifier(criterion='gini', max_depth=60)
    model = clf.fit(X_train, y_train)
    train_score = model.score(X_train, y_train)
    val_score = model.score(X_test, y_test)
    print('Decision Tree Train Accuracy: '+str(round(train_score*100,2))+'%')
    print('Decision Tree Validation Accuracy: '+str(round(val_score*100,2))+'%')